# Central Sulcus SNR and Information Capacity Mapping
Draws a simple central sulcus and simulates a depth electrode on both sides.

## Libraries and assigned values

In [ ]:
### Import libraries ###
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib import cm
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from os import path
import sys
sys.path.insert(0, path.join('.'))
from modules.leadfield_importer_manual_volume import FieldImporter

# Select dataset directory
jacedir = r"C:\Users\willi\Documents\NEI\Scripts\data"
fields_file = path.join(jacedir, 'DISC_30mm_p2-5Sm_MacChr.npz')

# Save folder
savedir = r"C:\Users\willi\Documents\NEI\SEPIO"


# Get lead fields
field_importer = FieldImporter()
field = field_importer.load(fields_file)
num_electrodes = np.shape(field_importer.fields)[4]
fields = field_importer.fields
scale = 0.5 # mm; voxel size


# Define dipoles
magnitude = 0.5e-9 # nAm
disc_noise = 4.1 # uV rms; typically 4.1 uV
seeg_noise = 2.3 # uV rms; typically 2.3 uV

## Draw section and determine values

In [ ]:
### Calculate curve and normal vectors ###
# A curve meant to emulate a deep sulcus (inner cortical surface)
xstart = -3
xstop = 3
xnum = 100
dx = (xstop-xstart)/xnum
x = np.linspace(xstart,xstop,xnum)
y = -7.5*np.cos((x**2)/3) - 7.5
vec = np.zeros((y.shape[0],2)) # [point,[-dy,dx]]
for i,xi in enumerate(x): # collect normal, unit vectors from y(x)
    vec[i] = np.array([(-y[i]+y[i-1]),dx])
    vec[i] *= 1/np.sqrt(vec[i,0]**2 + vec[i,1]**2)
vec[0] = vec[1] # Fix first vertical vector

# Plot
plt.plot(x,y)
plt.xlim([-7,7])
test = np.meshgrid(x,y)
plt.quiver(x,y,vec[:,0],vec[:,1])
plt.show()

In [ ]:
### Place devices and transform vector space for each
# Produces dippos and dipvec for calculating voltages

# Modify for device positions; [x,y] for the start and end of each device
devpos = np.array([
    [[-4.5,0],[-3,-14.5]], # Nearest x = 0
    [[4,0],[2.5,-14.5]]
])

# Translate & rotate vector locations to each device relative (still in mm)
dippos = np.zeros((2,2,y.shape[0])) # [device,x-y,point(x index)]
dippos[0] = np.array([x-np.mean(devpos[0],axis=0)[0],y-np.mean(devpos[0],axis=0)[1]])
dippos[1] = np.array([x-np.mean(devpos[1],axis=0)[0],y-np.mean(devpos[1],axis=0)[1]])
devangle = -np.pi/2 - np.array([np.arctan2((devpos[0,1,1]-devpos[0,0,1]),devpos[0,1,0]-devpos[0,0,0]),
                     np.arctan2((devpos[1,1,1]-devpos[1,0,1]),devpos[1,1,0]-devpos[1,0,0])])
R = np.array([[np.cos(devangle),-np.sin(devangle)],
              [np.sin(devangle),np.cos(devangle)]])
dippos[0] = np.dot(R[:,:,0],dippos[0])
dippos[1] = np.dot(R[:,:,1],dippos[1])
# Transfer position into leadfield space
dippos *= 1/scale
dippos += fields.shape[0]//2 # make measures relative to LF corner
dippos = dippos.astype('int')

# Rotate vectors for each device; [device,x-y,point(x index)]
dipvec = np.zeros((2,vec.shape[1],vec.shape[0]))
dipvec[0] = np.dot(R[:,:,0],vec.T)
dipvec[1] = np.dot(R[:,:,1],vec.T)
for i,xi in enumerate(x):
    dipvec[0,:,i] *= 1/np.sqrt(dipvec[0,0,i]**2 + dipvec[0,1,i]**2)
    dipvec[1,:,i] *= 1/np.sqrt(dipvec[1,0,i]**2 + dipvec[1,1,i]**2)
dipvec *= magnitude # Scale to dipole nAm magnitude

In [ ]:
### Calculate device voltage arrays
# Define depth to extend into z; should be lined up with scale
zstart = -8
zstop = 8
znum = int((zstop-zstart)/scale)+1
zmm = np.linspace(zstart,zstop,znum)
z = zmm/scale # scale z
z += fields.shape[0]//2 # set relative to LF corner
z = z.astype('int')

# voltage array of the shape [device,x,z,channel]; y is left out as it is defined by x
v = np.zeros((2,x.shape[0],z.shape[0],fields.shape[-1]))
# Swap axes and add a z axis to dipole vector; y -> z, new y is all 0
temp = np.zeros((2,3,x.shape[0])) # [device,3d vector,x position]
temp[:,0] += dipvec[:,0]
temp[:,2] += dipvec[:,1]
dipvec = np.copy(temp)
del temp

# Calculate voltages
for d in range(2): # d for device 0 or 1
    for e in range(fields.shape[-1]): # `e` for each electrode ID
        for ix,xi in enumerate(x):
            for iz,zi in enumerate(z):
                px = dippos[d,0,ix]
                py = dippos[d,1,ix]
                pz = zi
                LF = fields[px,pz,py,:,e]
                v[d,ix,iz,e] = np.dot(LF,dipvec[d,:,ix])

v *= 10**6 # Scale to uV units
v = np.nan_to_num(v) # fill zeros in place of NaN

## Settings for Plots

In [ ]:
SH = True # False -> Use SNR; True -> Use Shannon-Hartley information capacity
montage = False # Whether or not to apply montaging to find highest SNR combination
nrows = 2 # number of rows to combine into one SEEG ring; powers of 2; 2 is ~1mm, 4 is ~3 mm, 8 is ~7mm
snr_max = -1 # Max scale SNR to plot; -1 for none
snr_min = 1 # Anything below 1 goes to 0; Considers low signal to be indistinguishable from noise
sample_freq = 2000 # sampling frequency for DiSc
bw = sample_freq/2 # bandwidth for DiSc
color = cm.rainbow # MatPlotLib colormap
colordiff = cm.viridis #MPL colormap for difference plots

## DiSc SNR

In [ ]:
### Calculate SNR from the voltage on two devices ###
# Note v is of shape [device#,x,z,electrode#]

v2 = np.copy(v)
if montage: # Find SNR for montage
    # Find highest and lowest voltage for each spatial point
    v2max = np.max(v2,axis=3)
    v2min = np.min(v2,axis=3)
    # Largest difference regardless of device
    v2diff = np.max(v2max - v2min,axis=0)
    # Calculate SNR from difference and DiSc noise profile
    SNR = v2diff/disc_noise
    disc_SNR = np.copy(SNR)
else: # Find SNR for non-montage
    SNR = np.max(np.max(np.abs(v2),axis=3),axis=0)/disc_noise
    disc_SNR = np.copy(SNR)


## SEEG SNR

In [ ]:
### Convert voltages to SEEG ###
vseeg = np.zeros((v.shape[0],v.shape[1],v.shape[2],128//(8*nrows)))
for d in range(vseeg.shape[0]): # d for each device
    for s in range(v.shape[-1]): # s for each DiSc sensor
        column = s//16
        depth = s - column*16
        vseeg[d,:,:,depth//nrows] += v[d,:,:,s]

vseeg *= 1/(nrows*8) # divide by number of added sensors per ring

### Calculate SNR from the voltage on two devices ###
v2 = np.copy(vseeg)
if montage: # Find SNR for montage
    # Find highest and lowest voltage for each spatial point
    v2max = np.max(v2,axis=3)
    v2min = np.min(v2,axis=3)
    # Largest difference regardless of device
    v2diff = np.max(v2max - v2min,axis=0)
    # Calculate SNR from difference and DiSc noise profile
    SNR = v2diff/seeg_noise
    seeg_SNR = np.copy(SNR)
else: # Find SNR for non-montage
    SNR = np.max(np.max(np.abs(v2),axis=3),axis=0)/seeg_noise
    seeg_SNR = np.copy(SNR)

## Plot 3D Maps

In [ ]:
### Plot SNR map ###
# Plot
X, Z = np.meshgrid(x,zmm)
Y = np.repeat(y,znum).reshape((y.shape[0],znum)).T
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw=dict(projection='3d'), figsize=(12,6))
ls = LightSource(270,45)

seeg_SNR[seeg_SNR < snr_min] = 0
disc_SNR[disc_SNR < snr_min] = 0

if SH:
    sh_disc = bw*np.log2(disc_SNR.T + 1) # Information capacity; Shannon-Hartley
    sh_seeg = bw*np.log2(seeg_SNR.T + 1)

    vmin = min(sh_disc.min(), sh_seeg.min()) # Normalize datasets to the same scale 
    vmax = max(sh_disc.max(), sh_seeg.max())
    norm = Normalize(vmin=vmin, vmax=vmax)

    rgb1 = ls.shade(sh_disc,cmap=color,vert_exag=0.1,blend_mode='soft',norm=norm)
    surf1 = ax1.plot_surface(X,Z,Y,facecolors=rgb1,antialiased=True)

    rgb2 = ls.shade(sh_seeg,cmap=color,vert_exag=0.1,blend_mode='soft',norm=norm)
    surf2 = ax2.plot_surface(X,Z,Y,facecolors=rgb2,antialiased=True)

    cbar = fig.colorbar(cm.ScalarMappable(norm=norm,cmap=color),ax=[ax1,ax2],shrink=0.5,aspect=5,ticks=[0,0.5,1],orientation='vertical')
    ticks = [vmin,(vmin+vmax)/2,vmax]
    cbar.set_ticks(ticks)
    cbar.ax.set_yticklabels([str(int(vmin)), "Bits/s", str(int(vmax))])

else:
    if snr_max > 0:
        snr_plot_disc = np.where(disc_SNR > snr_max,snr_max,disc_SNR)
        snr_plot_seeg = np.where(seeg_SNR > snr_max,snr_max,seeg_SNR)
    else:
        snr_plot_disc = np.copy(disc_SNR)
        snr_plot_seeg = np.copy(seeg_SNR)

    vmin = min(snr_plot_disc.min(), snr_plot_seeg.min()) # Normalize datasets to the same scale 
    vmax = max(snr_plot_disc.max(), snr_plot_seeg.max())
    norm = Normalize(vmin=vmin, vmax=vmax)

    rgb1 = ls.shade(snr_plot_disc.T,cmap=color,vert_exag=0.1,blend_mode='soft',norm=norm)
    surf1 = ax1.plot_surface(X,Z,Y,facecolors=rgb1,antialiased=True)

    rgb2 = ls.shade(snr_plot_seeg.T,cmap=color,vert_exag=0.1,blend_mode='soft',norm=norm)
    surf2 = ax2.plot_surface(X,Z,Y,facecolors=rgb2,antialiased=True)

    cbar = fig.colorbar(cm.ScalarMappable(norm=norm,cmap=color),ax=[ax1,ax2],shrink=0.5,aspect=5,ticks=[0,0.5,1],orientation='vertical')
    ticks = [vmin,(vmin+vmax)/2,vmax]
    cbar.set_ticks(ticks)
    cbar.ax.set_yticklabels([str(int(vmin)), "Bits/s", str(int(vmax))])

fontprops = fm.FontProperties(size=10)
scalebar = AnchoredSizeBar(ax1.transData,0.038,'5 mm','lower center',
                           pad=0.1,color='black',frameon=False,
                           size_vertical=0.001,fontproperties=fontprops)
scalebar2 = AnchoredSizeBar(ax2.transData,0.038,'5 mm','lower center',
                           pad=0.1,color='black',frameon=False,
                           size_vertical=0.001,fontproperties=fontprops)
for ax in [ax1, ax2]:
    ax.set_xlim([-7.5, 7.5])
    ax.set_ylim([-7.5, 7.5])
    ax.axis('off')
    ax.grid(False)

ax1.add_artist(scalebar)
ax2.add_artist(scalebar2)
ax1.set_title("Central Sulcus Information Capacity DiSc")
ax2.set_title("Central Sulcus Information Capacity SEEG")

# plt.savefig(path.join(savedir,r"CSmap3dDiSc.pdf"),transparent=True)
plt.show()

## Plot 2D Heatmaps

In [ ]:
### Plot 2D heatmap of the disc data ###
plt.imshow(sh_disc, cmap = color, aspect = 1, extent = [0, 32, 0, 16],norm=norm)
# plt.savefig(path.join(savedir,r"CSmap2dDiSc.pdf"),transparent=True)
plt.show()

In [ ]:
### Plot 2D heatmap of the SEEG data ###
plt.imshow(sh_seeg, cmap = color, aspect = 1, extent = [0, 32, 0, 16],vmax=np.max(bw*np.log2(disc_SNR.T + 1)))
# plt.savefig(path.join(savedir,r"CSmap2dSEEG.pdf"),transparent=True)
plt.show()

## Difference between the two maps

In [ ]:
### Do the same but with the difference
SNR = disc_SNR - seeg_SNR

# Plot
X, Z = np.meshgrid(x,zmm)
Y = np.repeat(y,znum).reshape((y.shape[0],znum)).T
fig,ax = plt.subplots(subplot_kw=dict(projection='3d'))
ls = LightSource(270,45)
if SH:
    sh = bw*np.log2(disc_SNR.T + 1)-bw*np.log2(seeg_SNR.T + 1) # Information capacity; Shannon-Hartley
    rgb = ls.shade(sh,cmap=colordiff,vert_exag=0.1,blend_mode='soft')
    surf = ax.plot_surface(X,Z,Y,facecolors=rgb,antialiased=True)
    cbar = fig.colorbar(cm.ScalarMappable(cmap=colordiff),ax = plt.gca(),shrink=0.5,aspect=5,ticks=[0,0.5,1],orientation='vertical')
    cbar.ax.set_yticklabels([str(int(np.min(sh))),"Information Differential (Bits/s)",str(int(np.max(sh)))])
    plt.title("Information Capacity Differential: DiSc vs SEEG")
else:
    if snr_max > 0:
        snr_plot = np.where(SNR > snr_max,snr_max,SNR)
    else:
        snr_plot = np.copy(SNR)
    rgb = ls.shade(snr_plot.T,cmap=colordiff,vert_exag=0.1,blend_mode='soft')
    surf = ax.plot_surface(X,Z,Y,facecolors=rgb,antialiased=True)
    cbar = fig.colorbar(cm.ScalarMappable(cmap=colordiff),ax = plt.gca(),shrink=0.5,aspect=5,ticks=[0,0.5,1],orientation='vertical')
    cbar.ax.set_yticklabels([str(int(np.min(snr_plot))),"SNR",str(int(np.max(snr_plot)))])
    plt.title("SNR Differential: DiSc vs SEEG")
fontprops = fm.FontProperties(size=10)
scalebar = AnchoredSizeBar(ax.transData,0.038,'5 mm','lower center',
                           pad=0.1,color='black',frameon=False,
                           size_vertical=0.001,fontproperties=fontprops)
ax.add_artist(scalebar)
plt.xlim([-7.5,7.5])
plt.ylim([-7.5,7.5])
plt.axis('off')
ax.grid(False)
# plt.savefig(path.join(savedir,r"CSmap3dDiff.pdf"),transparent=True)
plt.show()

In [ ]:
### Plot 2D heatmap of the same data ###
plt.imshow(sh, cmap = colordiff, aspect = 1, extent = [0, 32, 0, 16])
# plt.savefig(path.join(savedir,r"CSmap2dDiff.pdf"),transparent=True)
plt.show()